In [1]:
import sys
sys.path.append('/dfs/scratch1/simran/tutorial/contextual-embeddings/')
import os
import random
import ujson
import argparse
import pandas as pd
import numpy as np
import pickle
import json
import csv
from collections import defaultdict, OrderedDict
import jsonlines
pd.set_option('display.max_colwidth', -1)
from tqdm import tqdm

/dfs/scratch1/simran/tutorial/bootleg-internal/.bootinternal/lib/python3.6/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
/dfs/scratch1/simran/tutorial/bootleg-internal/.bootinternal/lib/python3.6/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


# Utility functions

In [2]:
sys.path.append('/dfs/scratch1/simran/tutorial/bootleg-internal/notebooks')
sys.path.append('/dfs/scratch1/simran/tutorial/bootleg-internal/')
import utils_not_for_public as du
def load_rel_names_0905():
    return json.load(open("/dfs/scratch1/simran/bootleg_downstream/archive/wikidata_mappings/pid_names.json"))

es_0905, esp_0905, type_vocab_0905, typeid2typename_0905, type_vocab_wd_0905, typeid2typename_wd_0905, alias_qid_count_0905 = du.load_esp("wiki_0905", overwrite=False)     

import ujson
def load_json_file(filename):
    with open(filename, 'r') as f:
        contents = ujson.load(f)
    return contents


typename2title_wd_0905 = load_json_file("/dfs/scratch0/lorr1/bootleg/embs/wikidatatypeid_to_title_0905.json")
typeqid_to_id_0905 = json.load(open("/dfs/scratch0/lorr1/bootleg/embs/wikidata_to_typeid_0905.json"))
rel2id_0905 = '/dfs/scratch0/lorr1/bootleg/embs/relation_to_typeid_0905.json'
rel_to_name_0905 = load_rel_names_0905()

Reading in existing entity word bags from /dfs/scratch0/lorr1/data_prep/data/wiki0905/entity_db/entity_mappings/entity_bag_of_words
Reading in vocab from /dfs/scratch0/lorr1/data_prep/data/wiki0905/entity_db/entity_mappings/entity_all_words/all_words_vocab.marisa
Loaded entity symbols.
Loading alias qid counts from /dfs/scratch0/lorr1/data_prep/data/wiki0905/stats/alias_qid_traindata_withaugment.json
Found 506 hyena types
Found 26933 wikidata types
FINISHED LOADING IN 90.59796619415283


In [4]:


def get_relations_lst_0905(qid1, qid2):
    # If there are multiple relationships between two qids (both directions)
    rels = [esp_0905.get_relation_name(r) for r in esp_0905.get_all_relations(qid1, qid2)]
    rels = [rel_to_name_0905.get(rel, rel) for rel in rels]
    if rels:
        return rels[0]
    else:
        return ""

In [5]:
def load_mentions(file): 
    lines = []
    with jsonlines.open(file) as f: 
        for line in f: 
            new_line = {
                'id': line['id'],
                'sentence': line['sentence'],
                'aliases': line['aliases'], 
                'spans': line['spans'],
                'gold': line['gold'],
                'cand_probs': line['cand_probs'],
                'qids': line['qids'],
                'sent_idx_unq': line['sent_idx_unq'],
                'probs': line['probs'],
                'ctx_emb_ids': line['ctx_emb_ids'],
                'entity_ids': line['entity_ids']
            }
            lines.append(new_line)
    return pd.DataFrame(lines)

In [6]:
def get_span(span):
    span = span.split(":")
    span_s = int(span[0])
    span_e = int(span[1])
    return span_s, span_e

In [7]:
def load_mapping(file):
    lines = []
    with jsonlines.open(file) as f: 
        for line in f: 
            new_line = {
                'sentence': line['sentence'],
                'id': line['id']
            }
            lines.append(new_line)
    return pd.DataFrame(lines)

In [8]:
def load_dataset(file):
    lines = []
    with json.open(file) as f: 
        lines.append(new_line)
    return pd.DataFrame(lines)

# Load bootleg labels, mapping of labeled sentences to tacred ids, and tacred splits

In [13]:
data_dict = {
    'iclr':{'base_directory':'/dfs/scratch1/simran/tacred/tacred-relation-bootleg/dataset_bootleg_iclr_model/bootleg_092620/',
            'expt_directory': 'basic_full_sentences/',
           },
    '1007_full':{'base_directory':'/dfs/scratch1/simran/tacred/tacred-relation-bootleg/dataset_bootleg_ablations/full_expts/',
                 'expt_directory': 'basic_full_sentences/'},
    '1007_noty':{'base_directory':'/dfs/scratch1/simran/tacred/tacred-relation-bootleg/dataset_bootleg_ablations/noty_expts/',
                 'expt_directory': 'basic_full_sentences/'},
    '1007_clean_men':{'base_directory':'/dfs/scratch1/simran/tacred/tacred-relation-bootleg/dataset_bootleg_ablations/clean_men/',
                 'expt_directory': 'basic_full_sentences/'},
    '1007_clean':{'base_directory':'/dfs/scratch1/simran/tacred/tacred-relation-bootleg/dataset_bootleg_ablations/clean_expts/',
                 'expt_directory': 'basic_full_sentences/'},
    '1007_ft':{'base_directory':'/dfs/scratch1/simran/tacred/tacred-relation-bootleg/dataset_bootleg_ablations/ft_expts/',
                 'expt_directory': 'basic_full_sentences/'},
    '1007_clean_nowl':{'base_directory':'/dfs/scratch1/simran/tacred/tacred-relation-bootleg/dataset_bootleg_ablations/clean_nowl/',
                 'expt_directory': 'basic_full_sentences/'},
}

key = 'iclr'


In [41]:
base_directory = data_dict[key]['base_directory']
expt_directory = data_dict[key]['expt_directory']
boot_labels_file = "{}{}bootleg_labels.jsonl".format(base_directory, expt_directory)
bootleg_labels_df = load_mentions(boot_labels_file)
print(bootleg_labels_df.columns)
print(bootleg_labels_df.shape)

Index(['id', 'sentence', 'aliases', 'spans', 'gold', 'cand_probs', 'qids',
       'sent_idx_unq', 'probs', 'ctx_emb_ids', 'entity_ids'],
      dtype='object')
(106264, 11)


In [42]:
base_data = '/dfs/scratch1/simran/tacred/tacred-relation-bootleg/dataset_bootleg_iclr_model/tacred/base_data'

train_file = "{}/train.json".format(base_data)
with open(train_file) as train:
    df_train = json.load(train)
    df_train = pd.DataFrame.from_dict(df_train, orient='columns')
    print(df_train.shape)
    
dev_file = "{}/dev_rev.json".format(base_data)
with open(dev_file) as dev:
    df_dev = json.load(dev)
    df_dev = pd.DataFrame.from_dict(df_dev, orient='columns')
    print(df_dev.shape)
    
test_file = "{}/test_rev.json".format(base_data)
with open(test_file) as test:
    df_test = json.load(test)
    df_test = pd.DataFrame.from_dict(df_test, orient='columns')
    print(df_test.shape)


(68124, 14)
(22631, 14)
(15509, 14)


In [43]:
train_ids = df_train.loc[:,'id'].tolist()
dev_ids = df_dev.loc[:,'id'].tolist()
test_ids = df_test.loc[:,'id'].tolist()

In [44]:
# Filter bootleg QIDs 

In [45]:
import math
def entropy_candprobs(cand_probs):
    entropy = 0
    for p in cand_probs:
        if p > 0:
            entropy += (-p*math.log(p,2))
    return entropy

In [47]:
sub_test_df = bootleg_labels_df[bootleg_labels_df['id'].isin(test_ids)]
for ind, row in bootleg_labels_df.iterrows():
    for i in range(len(row['cand_probs'])):
        print(entropy_candprobs(row['cand_probs'][i]))
    print(row)
    if ind == 10:
        break

0.0
0.0
3.5208970776813384
0.0
1.7429673857213877
0.0
0.0
0.013588871907137886
1.1533277448001882
id              61b3a5c8c9a882dcfcd2                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

In [48]:
# for ind, row in bootleg_labels_df.iterrows():
#     for i in range(len(row['cand_probs'])):
#         print(entropy_candprobs(row['cand_probs'][i]))
#     print(row)
#     break

In [49]:
import unicodedata

def strip_accents(text):
    """
    Strip accents from input String.

    :param text: The input string.
    :type text: String.

    :returns: The processed String.
    :rtype: String.
    """
    try:
        text = unicode(text, 'utf-8')
    except (TypeError, NameError): # unicode is a default on python 3 
        pass
    text = unicodedata.normalize('NFD', text)
    text = text.encode('ascii', 'ignore')
    text = text.decode("utf-8")
    return str(text)

strip_accents('Beyoncé')

'Beyonce'

In [50]:
# professions list

f_profession = open('/dfs/scratch1/simran/tutorial/bootleg-internal/notebooks/type_categories/profession.txt', 'r')
profession_content = f_profession.read().split('\n')
profession_content.extend(['singer', 'politician', 'food manufacturer', 'actor', 'prisoner', 'journalist', 'poet',
                           'composer', 'songwriter', 'fashion designer', 'businessperson', 'singer-songwriter',
                           'producer', 'television producer', 'television actor', 'musician', 'film actor'])
print(type(profession_content))
print(profession_content)

<class 'list'>
['administrator', 'agent', 'analyst', 'architect', 'assistant', 'associate', 'buyer', 'clerk', 'comptroller', 'consultant', 'controller', 'coordinator', 'designer', 'developer', 'director', 'engineer', 'executive', 'facilitator', 'instructor', 'liason', 'manager', 'officer', 'orchestrator', 'planner', 'producer', 'representative', 'specialist', 'strategist', 'supervisor', 'technician', 'trainer', 'writer', 'athlete', 'coach', 'singer', 'politician', 'food manufacturer', 'actor', 'prisoner', 'journalist', 'poet', 'composer', 'songwriter', 'fashion designer', 'businessperson', 'singer-songwriter', 'producer', 'television producer', 'television actor', 'musician']


In [57]:
day_of_week = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
month = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

In [56]:
# BY ALIAS: 
alias = 'youxiaistan'
try:
    # To get qid candidates of an alias
    cands = esp.get_qid_cands(alias)
    print(f"Cands {cands}")
    print([es.get_title(qid) for qid in cands])
except:
    print("no candidates")
    pass


no candidates


In [71]:

filt_count = 0
for ind, row in tqdm(sub_test_df.iterrows()):
    row_id = row['id']
    if row_id in train_ids:
        tac_row = df_train[df_train['id']==row_id]
    elif row_id in test_ids:
        tac_row = df_test[df_test['id']==row_id]
    elif row_id in dev_ids:
        tac_row = df_dev[df_dev['id']==row_id]
        
    ner_tags = tac_row['stanford_ner'].item()
    relation = tac_row['relation'].item()
    
    qids = row['qids']
    probs = row['probs']
    spans = row['spans']
    aliases = row['aliases']
    cand_probs = row['cand_probs']
    ctx_emb_ids = row['ctx_emb_ids']
    entity_ids = row['entity_ids']
    gold = row['gold']
    
    tokens = row['sentence'].split(' ')
    
    # filter on people names with partial match
    idx_filt = []
    count = 0
    for qid,prob,span,alias,cand_prob in zip(qids, probs, spans, aliases, cand_probs):
        filt = 0
        ner_span = ner_tags[span[0]:span[1]]
#         alias_lst = alias.split(' ') # lowercase sentences
        try:
            alias_base = es_0905.get_title(qid)
            alias_base = strip_accents(alias_base)
            alias_lst = alias_base.split(' ')
            boot_types = du.get_qid_types(qid, esp_0905, typeid2typename_wd_0905, wikidata=True)
            boot_types = [typename2title_wd_0905[t] for t in boot_types]
            #print(boot_types)
        except:
            alias_base = ''
            alias_lst = []
            
        tok_base = ' '.join(tokens[span[0]:span[1]])
        tok_base = strip_accents(tok_base)
        tok_span = tok_base.split(' ')#tokens[span[0]:span[1]]
        
        # person filters
        if 'PERSON' in ner_span:
            if len(ner_span) < len(alias_lst) and entropy_candprobs(cand_prob) > 2:
                filt = 1
            elif len(ner_span) == len(alias_lst) and not all(tok in alias_lst for tok in tok_span):
                filt = 1
            elif entropy_candprobs(cand_prob) > 1.5:
                filt = 1 
            elif 'Company' in alias_base:
                filt = 1
                
                
        # filter entropy
        elif entropy_candprobs(cand_prob) > 2.5 and not all(tok in alias_lst for tok in tok_span) and not tok_base.isupper():
            filt = 1
            
            
        # filter lower case text that's classified as a named entity
        elif len(tok_span) == 1 and tok_span[0].islower() and len(alias_lst) > 1 and prob < 0.9:
            filt = 1
            
        # filter professions with low confidence
        elif any(ty in profession_content for ty in boot_types) and prob < 1.0:
            filt = 1
            
        # filter organizations
        elif 'Norris' in tok_base:
            filt = 1
        
        
        if 'Qiang' in tok_base:
                print(row['sentence'])
                print(relation)
                print(tok_span, alias_lst, prob, ner_span, entropy_candprobs(cand_prob))
                print()
        if not filt:
            idx_filt.append(count)
        else:
#             if 'country' in boot_types:# or 'politician' in boot_types:
#                 print(tok_span, alias_lst, prob, ner_span, entropy_candprobs(cand_prob))
#             if 'ORGANIZATION' in ner_span or 'LOCATION' in ner_span:
#                 print(tok_span, alias_lst, prob)
            filt_count += 1
        count += 1
        
    
    # construct final values
    cols = ['qids','probs','aliases','spans','cand_probs','ctx_emb_ids','entity_ids','gold']
    filtered_lists = [qids, probs, aliases, spans, cand_probs,  ctx_emb_ids,   entity_ids,   gold]
    for i in range(len(cols)):
        col = cols[i]
        filtered_list = filtered_lists[i]
        #bootleg_labels_df.at[ind, col] = [filtered_list[i] for i in idx_filt]

print(filt_count)

0it [00:00, ?it/s]/dfs/scratch1/simran/tutorial/bootleg-internal/.bootinternal/lib/python3.6/site-packages/ipykernel_launcher.py:11: FutureWarning: `item` has been deprecated and will be removed in a future version
  # This is added back by InteractiveShellApp.init_path()
/dfs/scratch1/simran/tutorial/bootleg-internal/.bootinternal/lib/python3.6/site-packages/ipykernel_launcher.py:12: FutureWarning: `item` has been deprecated and will be removed in a future version
  if sys.path[0] == '':
125it [00:00, 166.88it/s]

`` To get rich is glorious '' has been the mantra in booming communist China for 30 years , but few have embraced the slogan more vigorously than Wen Qiang , a leading law official in the country 's southwest .
no_relation
['Wen', 'Qiang'] ['Wen', 'Qiang'] 1.0 ['PERSON', 'PERSON'] 0.0



199it [00:01, 210.42it/s]

The verdict comes two days after Chongqing 's former police chief Wen Qiang went on trial for rape and accepting bribes in exchange for protecting the city 's extensive gang network .
per:charges
['Wen', 'Qiang'] ['Wen', 'Qiang'] 1.0 ['PERSON', 'PERSON'] 0.0



276it [00:01, 234.04it/s]

...

Chinese city says breaks up 14 gangs As part of the campaign , Wen Qiang , the previous deputy director of the city 's police -- a post he held for 16 years until last year -- has been detained in an internal investigation , Zhou said .
per:title
['Wen', 'Qiang'] ['Wen', 'Qiang'] 1.0 ['PERSON', 'PERSON'] 0.0



15441it [01:09, 202.67it/s]

In July , authorities executed Wen Qiang , a former top police and justice official , for charges including rape and taking bribes to protect criminal gangs .
per:cause_of_death
['Wen', 'Qiang'] ['Wen', 'Qiang'] 1.0 ['PERSON', 'PERSON'] 0.0

A municipal court in Chongqing convicted Xie Caiping -- sister-in-law of the former head of the city 's judiciary , Wen Qiang -- of a series of crimes linked to the gambling syndicate , said the verdict , posted on the court 's website .
no_relation
['Wen', 'Qiang'] ['Wen', 'Qiang'] 1.0 ['PERSON', 'PERSON'] 0.0



15509it [01:09, 222.89it/s]

Gang members arrested in China corruption case Earlier this month , China executed Wen Qiang , 55 , former director of the Chongqing Municipal Judicial Bureau for corruption charges involving bribery , rape , extortion and gang-related activities .
per:country_of_death
['Wen', 'Qiang'] ['Wen', 'Qiang'] 1.0 ['PERSON', 'PERSON'] 0.0

22577


In [ ]:
words = ['extra', 'models', 'aide', 'independent', 'Ashiq', 'Barker', 'Fisher', 'double', 'raiser', 'Tours', 'Norris']

# Run this to save popularity counts (e.g. in the Training Data)

In [22]:
# subj_counts = {}
# obj_counts = {}
# for ind, row in df_train.iterrows():
#     idx = row['id']
#     tokens = row['token']
#     ss, se = row['subj_start'], row['subj_end']
#     os_, oe = row['obj_start'], row['obj_end']
    
#     subj = ' '.join(tokens[ss:se+1]).lower()
#     obj = ' '.join(tokens[os_:oe+1]).lower()
    
#     if subj in subj_counts:
#         subj_counts[subj] = subj_counts[subj] + 1
#     else:
#         subj_counts[subj] = 1
        
#     if obj in obj_counts:
#         obj_counts[obj] = obj_counts[obj] + 1
#     else:
#         obj_counts[obj] = 1
        
# import json

# with open('/dfs/scratch1/simran/bootleg_downstream/error_outputs/subj_counts.json', 'w') as fp:
#     json.save(subj_counts, fp)
    
# with open('/dfs/scratch1/simran/bootleg_downstream/error_outputs/obj_counts.json', 'w') as fp:
#     json.save(obj_counts, fp)

# generate a remapping for the static entity ids (so that we don't need to use all 5.2 M)

In [ ]:
static_ent_ids_remap = []

for ind, row in bootleg_labels_df.iterrows():
    static_ids = row['entity_ids']
    for idx in static_ids:
        static_ent_ids_remap.append(idx)
assert -1 not in static_ent_ids_remap

In [165]:
print(len(static_ent_ids_remap))
print(len(set(static_ent_ids_remap)))

646904
60229


In [166]:
remap_static_ent_id_dict = {}
count = 0
for idx in set(static_ent_ids_remap):
    remap_static_ent_id_dict[idx] = count # old --> new
    count += 1

In [167]:
print(len(remap_static_ent_id_dict.keys()))

60229


# extract out the static entity embs based on the remapping

In [168]:
datafile = '{}{}/../static_embedding.npy'.format(base_directory, expt_directory)
out_dir = os.path.dirname('{}{}/'.format(base_directory, expt_directory))

if not os.path.exists(out_dir):
    os.makedirs(out_dir)

In [169]:
emb_data = np.load(datafile)

In [170]:
result_array = np.empty((len(set(static_ent_ids_remap)), 200))
from tqdm import tqdm

idx_lst = []
for idx_orig, idx_new in tqdm(remap_static_ent_id_dict.items()): # orig --> remap id
    idx_lst.append(idx_orig)
result_array = emb_data[np.array(idx_lst)] 
print(result_array.shape)
print(emb_data.shape)

100%|██████████| 60229/60229 [00:00<00:00, 790461.15it/s]

(60229, 256)
(5310043, 256)


In [151]:
# Sanity Checks
# for idx in idx_lst:
#     idx_orig = idx
#     idx_new = remap_static_ent_id_dict[idx_orig]
#     assert all(result_array[idx_new] - emb_data[idx_orig] == 0), ValueError('STATIC EMB REMAP ERROR!')
    
# idx_orig = idx_lst[30]
# idx_new = remap_static_ent_id_dict[idx_orig]
# print(idx_orig, idx_new, result_array[idx_new] - emb_data[idx_orig])
# print(type(result_array[idx_new][0]), type(emb_data[idx_orig][0]))


# num_rows = 10
# for ind, row in bootleg_labels_df.iterrows():
#     if num_rows == 0:
#         break
#     num_rows -= 1
    
#     example = row['sentence']
#     tokens = example.split(' ')
#     length = len(tokens) 
#     static_ids = row['entity_ids']
    
#     for idx in static_ids:
#         # static
#         static_emb_id = idx
#         static_emb_id_new = remap_static_ent_id_dict[static_emb_id] # map old id to new id
        
#         static_ent_emb_old = emb_data[static_emb_id]
#         static_ent_emb_new = result_array[static_emb_id_new]
#         print(static_ent_emb_old-static_ent_emb_new)
    

In [171]:
np.save('{DIR}/static_ent_emb_tacred.npy'.format(DIR=out_dir), result_array)

# Obtain the ent_id and entity_emb_id features to be used in tacred

In [23]:
ctx_emb_id_dict = {}
qid_dict = {}
emb_static_dict = {}
emb_static_dict_full = {}

ctx_emb_id_dict_first = {}
qid_dict_first = {}
emb_static_dict_first = {}
emb_static_dict_first_full = {}

for ind, row in bootleg_labels_df.iterrows():
    ctx_emb_ids = row['ctx_emb_ids']
    static_ids = row['entity_ids']
    qids = row['qids']
    spans = row['spans']
    
    # get sentence length
    example = row['sentence']
    tokens = example.split(' ')
    length = len(tokens)
    
    # initialize result datastructures
    ctx_emb_id_result = [-1] * length
    qid_result = ['UNK'] * length
    static_emb_id_result = [-1] * length
    
    ctx_emb_id_result_first = [-1] * length
    qid_result_first = ['UNK'] * length
    static_emb_id_result_first = [-1] * length
    
    for i in range(len(spans)):
        span = spans[i]
        start, end = span[0], span[1]
        span_len = end-start
        
        # contextual
        ctx_emb_id = ctx_emb_ids[i]
        ctx_emb_id_lst = [ctx_emb_id] * span_len
        ctx_emb_id_result[start:end] = ctx_emb_id_lst
        ctx_emb_id_result_first[start] = ctx_emb_id # just put it in the first lot
         
        # qids
        qid = qids[i]
        qid_lst = [qid] * span_len
        qid_result[start:end] = qid_lst
        qid_result_first[start] = qid
        
        # static
#         static_emb_id = static_ids[i]
#         static_emb_id = remap_static_ent_id_dict[static_emb_id] # map old id to new id
#         static_emb_id_lst = [static_emb_id] * span_len
#         static_emb_id_result[start:end] = static_emb_id_lst
#         static_emb_id_result_first[start] = static_emb_id
        
    idx = row['id']
    if idx in ctx_emb_id_dict:
        raise ValueError('duplicate indices!')
    
    ctx_emb_id_dict[idx] = ctx_emb_id_result
    qid_dict[idx] = qid_result
    emb_static_dict[idx] = static_emb_id_result
    
    ctx_emb_id_dict_first[idx] = ctx_emb_id_result_first
    qid_dict_first[idx] = qid_result_first
    emb_static_dict_first[idx] = static_emb_id_result_first



# Add the three features to the tacred data

### Training Data

In [24]:
df_train.shape

(68124, 14)

In [25]:
df_train["entity_emb_id"] = np.nan
df_train['entity_emb_id_first'] = np.nan
df_train['ent_id'] = np.nan
df_train['ent_id_first'] = np.nan
df_train['static_ent_emb_id'] = np.nan
df_train['static_ent_emb_id_first'] = np.nan

train_dict_ctx_emb_id = {}
train_dict_ctx_emb_id_first = {}
train_dict_qid = {}
train_dict_qid_first = {}
train_dict_static_emb_id = {}
train_dict_static_emb_id_first = {}

for ind, row in df_train.iterrows():
    idx = row['id']
    tokens = row['token']
    length = len(tokens)
    
    # initialize result datastructures
    ctx_emb_id_default = [-1] * length
    qid_default = ['UNK'] * length
    static_emb_id_default = [-1] * length
    
    # contextual
    if idx in ctx_emb_id_dict:
        train_dict_ctx_emb_id[idx] =  ctx_emb_id_dict[idx]
    else:
        train_dict_ctx_emb_id[idx] = ctx_emb_id_default
        
        
    if idx in ctx_emb_id_dict_first:
        train_dict_ctx_emb_id_first[idx] = ctx_emb_id_dict_first[idx]
    else:
        train_dict_ctx_emb_id_first[idx] = ctx_emb_id_default
        
    # qids
    if idx in qid_dict:
        train_dict_qid[idx] = qid_dict[idx]
    else:
        train_dict_qid[idx] = qid_default

        
    if idx in qid_dict_first:
        train_dict_qid_first[idx] = qid_dict_first[idx]
    else:
        train_dict_qid_first[idx] = qid_default
        
    # static 
    if idx in emb_static_dict:
        train_dict_static_emb_id[idx] = emb_static_dict[idx]
    else:
        train_dict_static_emb_id[idx] = static_emb_id_default
        
    if idx in emb_static_dict_first:
        train_dict_static_emb_id_first[idx] = emb_static_dict_first[idx]
    else:
        train_dict_static_emb_id_first[idx] = static_emb_id_default
        
assert len(train_dict_ctx_emb_id.keys()) == df_train.shape[0], print(len(train_dict_ctx_emb_id.keys()), df_train.shape[0])
assert len(train_dict_ctx_emb_id_first.keys()) == df_train.shape[0], print(len(train_dict_ctx_emb_id_first.keys()), df_train.shape[0])
assert len(train_dict_qid.keys()) == df_train.shape[0], print(len(train_dict_qid.keys()), df_train.shape[0])
assert len(train_dict_qid_first.keys()) == df_train.shape[0], print(len(train_dict_qid_first.keys()), df_train.shape[0])
assert len(train_dict_static_emb_id.keys()) == df_train.shape[0], print(len(train_dict_static_emb_id.keys()), df_train.shape[0])
assert len(train_dict_static_emb_id_first.keys()) == df_train.shape[0], print(len(train_dict_static_emb_id_first.keys()), df_train.shape[0])
df_train['static_ent_emb_id_first'] = df_train['id'].map(train_dict_static_emb_id_first)
df_train['entity_emb_id'] = df_train['id'].map(train_dict_ctx_emb_id)
df_train['entity_emb_id_first'] = df_train['id'].map(train_dict_ctx_emb_id_first)
df_train['ent_id'] = df_train['id'].map(train_dict_qid)
df_train['ent_id_first'] = df_train['id'].map(train_dict_qid_first)
df_train['static_ent_emb_id'] = df_train['id'].map(train_dict_static_emb_id)
df_train['static_ent_emb_id_first'] = df_train['id'].map(train_dict_static_emb_id_first)



In [26]:
df_train.shape

(68124, 20)

# Dev Data

In [27]:
df_dev.shape

(22631, 14)

In [28]:
df_dev["entity_emb_id"] = np.nan
df_dev['entity_emb_id_first'] = np.nan
df_dev['ent_id'] = np.nan
df_dev['ent_id_first'] = np.nan
df_dev['static_ent_emb_id'] = np.nan
df_dev['static_ent_emb_id_first'] = np.nan

dev_dict_ctx_emb_id = {}
dev_dict_ctx_emb_id_first = {}
dev_dict_qid = {}
dev_dict_qid_first = {}
dev_dict_static_emb_id = {}
dev_dict_static_emb_id_first = {}

for ind, row in df_dev.iterrows():
    idx = row['id']
    tokens = row['token']
    length = len(tokens)
    
    # initialize result datastructures
    ctx_emb_id_default = [-1] * length
    qid_default = ['UNK'] * length
    static_emb_id_default = [-1] * length
    
    # contextual
    if idx in ctx_emb_id_dict:
        dev_dict_ctx_emb_id[idx] =  ctx_emb_id_dict[idx]
    else:
        dev_dict_ctx_emb_id[idx] = ctx_emb_id_default
        
        
    if idx in ctx_emb_id_dict_first:
        dev_dict_ctx_emb_id_first[idx] = ctx_emb_id_dict_first[idx]
    else:
        dev_dict_ctx_emb_id_first[idx] = ctx_emb_id_default
        
    # qids
    if idx in qid_dict:
        dev_dict_qid[idx] = qid_dict[idx]
    else:
        dev_dict_qid[idx] = qid_default

        
    if idx in qid_dict_first:
        dev_dict_qid_first[idx] = qid_dict_first[idx]
    else:
        dev_dict_qid_first[idx] = qid_default
        
    # static 
    if idx in emb_static_dict:
        dev_dict_static_emb_id[idx] = emb_static_dict[idx]
    else:
        dev_dict_static_emb_id[idx] = static_emb_id_default
        
    if idx in emb_static_dict_first:
        dev_dict_static_emb_id_first[idx] = emb_static_dict_first[idx]
    else:
        dev_dict_static_emb_id_first[idx] = static_emb_id_default
        
assert len(dev_dict_ctx_emb_id.keys()) == df_dev.shape[0], print(len(dev_dict_ctx_emb_id.keys()), df_dev.shape[0])
assert len(dev_dict_ctx_emb_id_first.keys()) == df_dev.shape[0], print(len(dev_dict_ctx_emb_id_first.keys()), df_dev.shape[0])
assert len(dev_dict_qid.keys()) == df_dev.shape[0], print(len(dev_dict_qid.keys()), df_dev.shape[0])
assert len(dev_dict_qid_first.keys()) == df_dev.shape[0], print(len(dev_dict_qid_first.keys()), df_dev.shape[0])
assert len(dev_dict_static_emb_id.keys()) == df_dev.shape[0], print(len(dev_dict_static_emb_id.keys()), df_dev.shape[0])
assert len(dev_dict_static_emb_id_first.keys()) == df_dev.shape[0], print(len(dev_dict_static_emb_id_first.keys()), df_dev.shape[0])
df_dev['entity_emb_id'] = df_dev['id'].map(dev_dict_ctx_emb_id)
df_dev['entity_emb_id_first'] = df_dev['id'].map(dev_dict_ctx_emb_id_first)
df_dev['ent_id'] = df_dev['id'].map(dev_dict_qid)
df_dev['ent_id_first'] = df_dev['id'].map(dev_dict_qid_first)
df_dev['static_ent_emb_id'] = df_dev['id'].map(dev_dict_static_emb_id)
df_dev['static_ent_emb_id_first'] = df_dev['id'].map(dev_dict_static_emb_id_first)

In [29]:
df_dev.shape

(22631, 20)

# Test Data

In [30]:
df_test.shape

(15509, 14)

In [31]:
df_test["entity_emb_id"] = np.nan
df_test['entity_emb_id_first'] = np.nan
df_test['ent_id'] = np.nan
df_test['ent_id_first'] = np.nan
df_test['static_ent_emb_id'] = np.nan
df_test['static_ent_emb_id_first'] = np.nan

test_dict_ctx_emb_id = {}
test_dict_ctx_emb_id_first = {}
test_dict_qid = {}
test_dict_qid_first = {}
test_dict_static_emb_id = {}
test_dict_static_emb_id_first = {}

for ind, row in df_test.iterrows():
    idx = row['id']
    tokens = row['token']
    length = len(tokens)
    
    # initialize result datastructures
    ctx_emb_id_default = [-1] * length
    qid_default = ['UNK'] * length
    static_emb_id_default = [-1] * length
    
    # contextual
    if idx in ctx_emb_id_dict:
        test_dict_ctx_emb_id[idx] =  ctx_emb_id_dict[idx]
    else:
        test_dict_ctx_emb_id[idx] = ctx_emb_id_default
        
        
    if idx in ctx_emb_id_dict_first:
        test_dict_ctx_emb_id_first[idx] = ctx_emb_id_dict_first[idx]
    else:
        test_dict_ctx_emb_id_first[idx] = ctx_emb_id_default
        
    # qids
    if idx in qid_dict:
        test_dict_qid[idx] = qid_dict[idx]
    else:
        test_dict_qid[idx] = qid_default

        
    if idx in qid_dict_first:
        test_dict_qid_first[idx] = qid_dict_first[idx]
    else:
        test_dict_qid_first[idx] = qid_default
        
    # static 
    if idx in emb_static_dict:
        test_dict_static_emb_id[idx] = emb_static_dict[idx]
    else:
        test_dict_static_emb_id[idx] = static_emb_id_default
        
    if idx in emb_static_dict_first:
        test_dict_static_emb_id_first[idx] = emb_static_dict_first[idx]
    else:
        test_dict_static_emb_id_first[idx] = static_emb_id_default

assert len(test_dict_qid.keys()) == df_test.shape[0], print(len(test_dict_qid.keys()), df_test.shape[0])
assert len(test_dict_static_emb_id.keys()) == df_test.shape[0], print(len(test_dict_static_emb_id.keys()), df_test.shape[0])
assert len(test_dict_qid_first.keys()) == df_test.shape[0], print(len(test_dict_qid_first.keys()), df_test.shape[0])
assert len(test_dict_ctx_emb_id_first.keys()) == df_test.shape[0], print(len(test_dict_ctx_emb_id_first.keys()), df_test.shape[0])
assert len(test_dict_ctx_emb_id.keys()) == df_test.shape[0], print(len(test_dict_ctx_emb_id.keys()), df_test.shape[0])
assert len(test_dict_static_emb_id_first.keys()) == df_test.shape[0], print(len(test_dict_static_emb_id_first.keys()), df_test.shape[0])
df_test['entity_emb_id'] = df_test['id'].map(test_dict_ctx_emb_id)
df_test['entity_emb_id_first'] = df_test['id'].map(test_dict_ctx_emb_id_first)
df_test['ent_id'] = df_test['id'].map(test_dict_qid)
df_test['ent_id_first'] = df_test['id'].map(test_dict_qid_first)
df_test['static_ent_emb_id'] = df_test['id'].map(test_dict_static_emb_id)
df_test['static_ent_emb_id_first'] = df_test['id'].map(test_dict_static_emb_id_first)

In [32]:
df_test.shape

(15509, 20)

# Sanity check the static ids

In [33]:
# for ind, row in df_dev.iterrows():
#     if 'Millipore' in row['token']:
#         print(row[['token', 'entity_emb_id', 'entity_emb_id_first', 'ent_id', 'ent_id_first', 'static_ent_emb_id', 'static_ent_emb_id_first', 'subj_start', 'obj_start']])

In [34]:
# # sanity check
# pd.set_option('display.max_colwidth', -1)
# for ind, row in bootleg_labels_df.iterrows():
#     if 'Millipore' in row['sentence']:
#         print(row[['sentence', 'entity_ids']])

# Save the appended data

In [36]:
out_dir = os.path.dirname('{}{}/filt_1015_v1/'.format(base_directory, expt_directory))
print(out_dir)

/dfs/scratch1/simran/tacred/tacred-relation-bootleg/dataset_bootleg_iclr_model/bootleg_092620/basic_full_sentences//filt_1015_v1


In [38]:
train_out = df_train.to_json(r'{}/train_ent.json'.format(out_dir),orient='records')

In [39]:
dev_out = df_dev.to_json(r'{}/dev_rev_ent.json'.format(out_dir),orient='records')

In [40]:
test_out = df_test.to_json(r'{}/test_rev_ent.json'.format(out_dir),orient='records')

In [65]:
# SCRAP

In [66]:
# # MAP IF NEEDED
# alldata_file = "{}all_tacred_data_glovenormalized_mapping.jsonl".format(directory)

# alldata_df = load_mapping(alldata_file)
# print(len(alldata_df))

# alldata_dict = {}
# for ind, row in alldata_df.iterrows():
#     if row['sentence'] in alldata_dict:
#         alldata_dict[row['sentence']].append(row['id'])
#     else:
#         alldata_dict[row['sentence']] = [row['id']]
# len(alldata_dict)


# mapping_dict = {}
# for i, row in bootleg_labels_df.iterrows():
#     sent = row['sentence']
#     uq_id = row['sent_idx_unq']
#     if sent in alldata_dict:
#         map_id = alldata_dict[sent].pop()
#         mapping_dict[uq_id] = map_id
#         if not alldata_dict[sent]:
#             del alldata_dict[sent]
#     else:
#         print("AHHH")
            
# print(len(mapping_dict.keys()))    
# bootleg_labels_df['id'] = bootleg_labels_df['sent_idx_unq'].map(mapping_dict)
# print(bootleg_labels_df.shape)